In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/dacon_cv/
!ls

/content/drive/MyDrive/dacon_cv
'제_2회 컴퓨터 비전 학습 경진대회 베이스라인 코드.ipynb'
 ASL
 converT2coco.ipynb
 dirty_mnist
 dirty_mnist_2nd_answer.csv
 dirty_mnist_2nd.zip
 dirty_mnist_answer.csv
 dirty_mnist.zip
 mnist_data
 sample_submission.csv
 test_dirty_mnist
 test_dirty_mnist_2nd.zip
 test.json


In [ ]:
!nvidia-smi

Fri Feb 19 18:04:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!rm -rf dirty_mnist/ 
!mkdir dirty_mnist/
!ls

'제_2회 컴퓨터 비전 학습 경진대회 베이스라인 코드.ipynb'
 ASL
 converT2coco.ipynb
 dirty_mnist
 dirty_mnist_2nd_answer.csv
 dirty_mnist_2nd.zip
 dirty_mnist_answer.csv
 dirty_mnist.zip
 mnist_data
 sample_submission.csv
 test_dirty_mnist
 test_dirty_mnist_2nd.zip
 test.json


In [ ]:
!unzip -q dirty_mnist_2nd.zip -d dirty_mnist/
!ls

'제_2회 컴퓨터 비전 학습 경진대회 베이스라인 코드.ipynb'
 ASL
 converT2coco.ipynb
 dirty_mnist
 dirty_mnist_2nd_answer.csv
 dirty_mnist_2nd.zip
 dirty_mnist_answer.csv
 dirty_mnist.zip
 mnist_data
 sample_submission.csv
 test_dirty_mnist
 test_dirty_mnist_2nd.zip
 test.json


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import shutil
import os
import json

In [ ]:
namelist = os.listdir('./dirty_mnist/')
answer = pd.read_csv('dirty_mnist_answer.csv')
train, val = train_test_split(namelist, test_size=0.2, random_state=23)
train_num = [int(t[:5]) for t in train]
val_num = [int(v[:5]) for v in val]
train_ohe = [True if i in train_num else False for i in range(50000)]
val_ohe = [True if i in val_num else False for i in range(50000)]
train_ans = answer[train_ohe]
val_ans = answer[val_ohe]

In [ ]:
%cd ./dirty_mnist/

/content/drive/My Drive/dacon_cv/dirty_mnist


In [ ]:
os.makedirs('./dirty_mnist_train', exist_ok=True)
os.makedirs('./dirty_mnist_val', exist_ok=True)
for t in train:
    shutil.move(t, './dirty_mnist_train')
for v in val:
    shutil.move(v, './dirty_mnist_val')

In [ ]:
def csv_to_coco(df, json_name):
    dic = {
        "info": {
            "description": "DACON DIRTY MNIST DATASET",
            "url": "https://oranz.tistory.com/",
            "version": "1.0",
            "year": 2021,
            "contributor": "DACON",
            "date_created": "2021/02/18"
        },
        "licenses": [{
            "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/",
            "id": 1,
            "name": "Attribution-NonCommercial-ShareAlike License"
        }],
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    for i in range(1, 27):
        dic["categories"].append({"supercategory": "alphabet", "id": i, "name": chr(i + 96)})
        
    anno_id = 1
    for r in range(len(df)):
        image_id = int(df.iloc[r, 0]) # cast to int for JSON serialization
        name = str(image_id)
        name = '0' * (5 - len(name)) + name + '.png'
        dic["images"].append({
            "license": 1,
            "file_name": name,
            "coco_url": "",
            "height": 256,
            "width": 256,
            "date_captured": "2020-05-19 23:03:57",
            "flickr_url": "",
            "id": image_id
        })
        
        for c in range(1, 27):
            if df.iloc[r, c] == 1:
                dic["annotations"].append({
                    "segmentation": [],
                    "area": 232,
                    "iscrowd": 0,
                    "image_id": image_id,
                    "bbox": [1, 5, 4, 9],
                    "category_id": c,
                    "id": anno_id
                })
                anno_id += 1
    with open(json_name, "w") as json_file:
        json.dump(dic, json_file)

In [ ]:
csv_to_coco(val_ans, "val.json")
csv_to_coco(train_ans, "train.json")

In [ ]:
!pip install inplace-abn
!pip install randaugment

     |████████████████████████████████| 143kB 10.6MB/s 
  Created wheel for inplace-abn: filename=inplace_abn-1.1.0-cp36-cp36m-linux_x86_64.whl size=2752485 sha256=99c13f92e1e89721307f9907d3ffc502eaf2e9b815759ae08a56b20f8c2c1661
  Stored in directory: /root/.cache/pip/wheels/90/e6/ce/baadcff0441c600caa5874d4d3322a7909e724fb7abab21a15
Successfully built inplace-abn


In [ ]:
cd ../ASL

/content/drive/My Drive/dacon_cv/ASL


In [ ]:
import argparse
import easydict
import torch
import torch.nn.parallel
import torch.optim
import torch.utils.data.distributed
import torchvision.transforms as transforms
import os
from torch.optim import lr_scheduler
from src.helper_functions.helper_functions import mAP, CocoDetection, CutoutPIL, ModelEma, add_weight_decay
from src.models import create_model
from src.loss_functions.losses import AsymmetricLoss
from randaugment import RandAugment
from torch.cuda.amp import GradScaler, autocast

parser = argparse.ArgumentParser(description='PyTorch MS_COCO Training')
parser.add_argument('data', metavar='DIR', help='path to dataset', default='../dirty_mnist')
parser.add_argument('--lr', default=1e-4, type=float)
parser.add_argument('--model-name', default='tresnet_m')
parser.add_argument('--model-path', default='./tresnet_m.pth', type=str)
parser.add_argument('--num-classes', default=26)
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                    help='number of data loading workers (default: 16)')
parser.add_argument('--image-size', default=224, type=int,
                    metavar='N', help='input image size (default: 448)')
parser.add_argument('--thre', default=0.8, type=float,
                    metavar='N', help='threshold value')
parser.add_argument('-b', '--batch-size', default=256, type=int,
                    metavar='N', help='mini-batch size (default: 16)')
parser.add_argument('--print-freq', '-p', default=64, type=int,
                    metavar='N', help='print frequency (default: 64)')


def main():
    args = easydict.EasyDict({ "data": "../dirty_mnist", 
                              "lr": 23e-5, 
                              "model_name": "tresnet_m", 
                              "model_path": "./tresnet_m.pth",
                              "num_classes": 26, 
                              "workers": 4, 
                              "image_size": 224, 
                              "thre": 0.8,
                              "batch_size": 256,
                              "print_freq": 64
                              })
    args.do_bottleneck_head = False

    # Setup model
    print('creating model...')
    model = create_model(args).cuda()
    if args.model_path:  # make sure to load pretrained ImageNet model
        state = torch.load(args.model_path, map_location='cpu')
        filtered_dict = {k: v for k, v in state['model'].items() if
                         (k in model.state_dict() and 'head.fc' not in k)}
        model.load_state_dict(filtered_dict, strict=False)
    print('done\n')

    # COCO Data loading
    instances_path_val = os.path.join(args.data, 'val.json')
    instances_path_train = os.path.join(args.data, 'train.json')
    # data_path_val = args.data
    # data_path_train = args.data
    data_path_val   = f'{args.data}/dirty_mnist_val'    # args.data
    data_path_train = f'{args.data}/dirty_mnist_train'  # args.data
    val_dataset = CocoDetection(data_path_val,
                                instances_path_val,
                                transforms.Compose([
                                    transforms.Resize((args.image_size, args.image_size)),
                                    transforms.ToTensor(),
                                    # normalize, # no need, toTensor does normalization
                                ]))
    train_dataset = CocoDetection(data_path_train,
                                  instances_path_train,
                                  transforms.Compose([
                                      transforms.Resize((args.image_size, args.image_size)),
                                      CutoutPIL(cutout_factor=0.5),
                                      RandAugment(),
                                      transforms.ToTensor(),
                                      # normalize,
                                  ]))
    print("len(val_dataset)): ", len(val_dataset))
    print("len(train_dataset)): ", len(train_dataset))

    # Pytorch Data loader
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=args.batch_size, shuffle=True,
        num_workers=args.workers, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=args.batch_size, shuffle=False,
        num_workers=args.workers, pin_memory=False)

    # Actuall Training
    train_multi_label_coco(model, train_loader, val_loader, args.lr)


def train_multi_label_coco(model, train_loader, val_loader, lr):
    ema = ModelEma(model, 0.9997)  # 0.9997^641=0.82

    # set optimizer
    Epochs = 80
    Stop_epoch = 40
    weight_decay = 1e-4
    criterion = AsymmetricLoss(gamma_neg=2, gamma_pos=1, clip=0)
    parameters = add_weight_decay(model, weight_decay)
    optimizer = torch.optim.Adam(params=parameters, lr=lr, weight_decay=0)  # true wd, filter_bias_and_bn
    steps_per_epoch = len(train_loader)
    scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=lr, steps_per_epoch=steps_per_epoch, epochs=Epochs,
                                        pct_start=0.2)

    highest_mAP = 0
    trainInfoList = []
    scaler = GradScaler()
    for epoch in range(Epochs):
        for i, (inputData, target) in enumerate(train_loader):
            inputData = inputData.cuda()
            target = target.cuda()  # (batch,3,num_classes)
            target = target.max(dim=1)[0]
            with autocast():  # mixed precision
                output = model(inputData).float()  # sigmoid will be done in loss !
            loss = criterion(output, target)
            model.zero_grad()

            scaler.scale(loss).backward()
            # loss.backward()

            scaler.step(optimizer)
            scaler.update()
            # optimizer.step()

            scheduler.step()

            ema.update(model)
            # store information
            if i % 50 == 0:
                trainInfoList.append([epoch, i, loss.item()])
                print('Epoch [{}/{}], Step [{}/{}], LR {:.1e}, Loss: {:.1f}'
                      .format(epoch, Epochs, str(i).zfill(3), str(steps_per_epoch).zfill(3),
                              scheduler.get_last_lr()[0], \
                              loss.item()))

        try:
            torch.save(model.state_dict(), os.path.join(
                'model_asl/', 'model-{}-{}.pth'.format(epoch + 1, i + 1)))
        except:
            print("model per epoch is not being saved.")
            pass

        model.eval()
        mAP_score = validate_multi(val_loader, model, ema)
        model.train()
        if mAP_score > highest_mAP:
            highest_mAP = mAP_score
            try:
                torch.save(model.state_dict(), os.path.join(
                    'model_asl/', 'model-highest.pth'))
            except:
                print("best model is not being saved.")
                pass
        print('current_mAP = {:.2f}, highest_mAP = {:.2f}\n'.format(mAP_score, highest_mAP))


def validate_multi(val_loader, model, ema_model):
    print("starting validation")
    Sig = torch.nn.Sigmoid()
    preds_regular = []
    preds_ema = []
    targets = []
    for i, (input, target) in enumerate(val_loader):
        target = target
        target = target.max(dim=1)[0]
        # compute output
        with torch.no_grad():
            with autocast():
                output_regular = Sig(model(input.cuda())).cpu()
                output_ema = Sig(ema_model.module(input.cuda())).cpu()

        # for mAP calculation
        preds_regular.append(output_regular.cpu().detach())
        preds_ema.append(output_ema.cpu().detach())
        targets.append(target.cpu().detach())

    mAP_score_regular = mAP(torch.cat(targets).numpy(), torch.cat(preds_regular).numpy())
    mAP_score_ema = mAP(torch.cat(targets).numpy(), torch.cat(preds_ema).numpy())
    print("mAP score regular {:.2f}, mAP score EMA {:.2f}".format(mAP_score_regular, mAP_score_ema))
    return max(mAP_score_regular, mAP_score_ema)


main()

creating model...
done

loading annotations into memory...
Done (t=0.53s)
creating index...
index created!
loading annotations into memory...
Done (t=2.20s)
creating index...
index created!
len(val_dataset)):  10000
len(train_dataset)):  40000


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch [0/80], Step [000/157], LR 9.2e-06, Loss: 1742.2
Epoch [0/80], Step [050/157], LR 9.4e-06, Loss: 1712.3
Epoch [0/80], Step [100/157], LR 1.0e-05, Loss: 1692.2
Epoch [0/80], Step [150/157], LR 1.1e-05, Loss: 1681.7
starting validation
mAP score regular 46.71, mAP score EMA 46.27
current_mAP = 46.71, highest_mAP = 46.71

Epoch [1/80], Step [000/157], LR 1.1e-05, Loss: 1681.0
Epoch [1/80], Step [050/157], LR 1.3e-05, Loss: 1678.6
Epoch [1/80], Step [100/157], LR 1.5e-05, Loss: 1674.1
Epoch [1/80], Step [150/157], LR 1.7e-05, Loss: 1679.1
starting validation
mAP score regular 47.24, mAP score EMA 46.27
current_mAP = 47.24, highest_mAP = 47.24

Epoch [2/80], Step [000/157], LR 1.8e-05, Loss: 1674.5
Epoch [2/80], Step [050/157], LR 2.1e-05, Loss: 1672.8
Epoch [2/80], Step [100/157], LR 2.4e-05, Loss: 1675.7
Epoch [2/80], Step [150/157], LR 2.7e-05, Loss: 1674.0
starting validation
mAP score regular 48.35, mAP score EMA 46.29
current_mAP = 48.35, highest_mAP = 48.35

Epoch [3/80], Step 

In [ ]:
import torch
from src.helper_functions.helper_functions import mAP, CocoDetection, CutoutPIL, ModelEma, add_weight_decay
from src.loss_functions.losses import AsymmetricLoss, AsymmetricLossOptimized
from src.models import create_model
import argparse
import numpy as np
import pandas as pd


# parsing args
args = easydict.EasyDict({ "model_path": "./models/model-highest.pth",
                          "model_name": "tresnet_m",
                          "workers": 4, 
                          "dataset_type": "MS-COCO", 
                          "th": 0.8,
                          "num_classes": 26
                          })
instances_path_test = '/content/drive/My Drive/dacon_cv/test.json'
data_path_test = '/content/drive/My Drive/dacon_cv/test_dirty_mnist'
test_dataset = CocoDetection(data_path_test,
                            instances_path_test,
                            transforms.Compose([
                                transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                            ]))

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=128, shuffle=False,
    num_workers=4, pin_memory=True, drop_last=False)

predictions_list = []
# 배치 단위로 추론
prediction_df = pd.read_csv("/content/drive/My Drive/dacon_cv/sample_submission.csv")
prediction_array = np.zeros([prediction_df.shape[0],
                            prediction_df.shape[1]- 1])


                    
print("test dset: ", test_dataset)

# setup model
print('creating and loading the model...')
state = torch.load(args.model_path, map_location='cpu')
model = create_model(args).cuda()
model.load_state_dict(state, strict=True)
for idx, sample in enumerate(test_data_loader):
    with torch.no_grad():
        # 추론
        model.eval()
        images = sample[0].cuda()
        probs  = model(images)
        probs = probs.cpu().detach().numpy()
        preds = (probs > 0.7)

        # 예측 결과를 
        # prediction_array에 입력
        batch_index = 128 * idx
        prediction_array[batch_index: batch_index + images.shape[0],:] = preds.astype(int)


loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
test dset:  Dataset CocoDetection
    Number of datapoints: 5000
    Root location: /content/drive/My Drive/dacon_cv/test_dirty_mnist
creating and loading the model...


In [ ]:
sample_submission = pd.read_csv("../sample_submission.csv")
sample_submission.iloc[:,1:] = prediction_array
sample_submission.to_csv("prediction.csv", index = False)
sample_submission

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,50001,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,50002,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,50003,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,50004,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4996,54996,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4997,54997,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4998,54998,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
import os
from copy import deepcopy
import random
import time
from copy import deepcopy

import numpy as np
from PIL import Image
from torchvision import datasets as datasets
import torch
from PIL import ImageDraw
from pycocotools.coco import COCO

class CocoDetection(datasets.coco.CocoDetection):
    def __init__(self, root, annFile, transform=None, target_transform=None):
        self.root = root
        self.coco = COCO(annFile)
        self.ids = list(self.coco.imgToAnns.keys())
        print(self.ids)
        self.transform = transform
        self.target_transform = target_transform
        self.cat2cat = dict()
        for cat in self.coco.cats.keys():
            self.cat2cat[cat] = len(self.cat2cat)
        # print(self.cat2cat)

    def __getitem__(self, index):
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        target = coco.loadAnns(ann_ids)

        output = torch.zeros((3, 26), dtype=torch.long)
        for obj in target:
            if obj['area'] < 32 * 32:
                output[0][self.cat2cat[obj['category_id']]] = 1
            elif obj['area'] < 96 * 96:
                output[1][self.cat2cat[obj['category_id']]] = 1
            else:
                output[2][self.cat2cat[obj['category_id']]] = 1
        target = output

        path = coco.loadImgs(img_id)[0]['file_name']
        img = Image.open(os.path.join(self.root, path)).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)
        return img, target

test_dataset = CocoDetection(data_path_test,
                            instances_path_test,
                            transforms.Compose([
                                transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                            ]))
test_dataset

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
[50000, 50001, 50002, 50003, 50004, 50005, 50006, 50007, 50008, 50009, 50010, 50011, 50012, 50013, 50014, 50015, 50016, 50017, 50018, 50019, 50020, 50021, 50022, 50023, 50024, 50025, 50026, 50027, 50028, 50029, 50030, 50031, 50032, 50033, 50034, 50035, 50036, 50037, 50038, 50039, 50040, 50041, 50042, 50043, 50044, 50045, 50046, 50047, 50048, 50049, 50050, 50051, 50052, 50053, 50054, 50055, 50056, 50057, 50058, 50059, 50060, 50061, 50062, 50063, 50064, 50065, 50066, 50067, 50068, 50069, 50070, 50071, 50072, 50073, 50074, 50075, 50076, 50077, 50078, 50079, 50080, 50081, 50082, 50083, 50084, 50085, 50086, 50087, 50088, 50089, 50090, 50091, 50092, 50093, 50094, 50095, 50096, 50097, 50098, 50099, 50100, 50101, 50102, 50103, 50104, 50105, 50106, 50107, 50108, 50109, 50110, 50111, 50112, 50113, 50114, 50115, 50116, 50117, 50118, 50119, 50120, 50121, 50122, 50123, 50124, 50125, 50126, 50127, 50128, 50129, 50130,

Dataset CocoDetection
    Number of datapoints: 5000
    Root location: /content/drive/My Drive/dacon_cv/test_dirty_mnist

In [ ]:
%cd ../
!ls

/content/drive/MyDrive/dacon_cv
'제_2회 컴퓨터 비전 학습 경진대회 베이스라인 코드.ipynb'
 ASL
 converT2coco.ipynb
 data.zip
 dirty_mnist
 dirty_mnist_2nd_answer.csv
 dirty_mnist_2nd.zip
 dirty_mnist_answer.csv
 dirty_mnist.zip
 mnist_data
 sample_submission.csv
 test_dirty_mnist
 test_dirty_mnist_2nd.zip
 test.json


In [ ]:
from random import randint

def csv_to_coco(df, json_name):
    dic = {
        "info": {
            "description": "DACON DIRTY MNIST DATASET",
            "url": "https://oranz.tistory.com/",
            "version": "1.0",
            "year": 2021,
            "contributor": "DACON",
            "date_created": "2021/02/18"
        },
        "licenses": [{
            "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/",
            "id": 1,
            "name": "Attribution-NonCommercial-ShareAlike License"
        }],
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    for i in range(1, 27):
        dic["categories"].append({"supercategory": "alphabet", "id": i, "name": chr(i + 96)})
        
    anno_id = 1
    for r in range(len(df)):
        image_id = int(df.iloc[r, 0]) # cast to int for JSON serialization
        name = str(image_id)
        name = '0' * (5 - len(name)) + name + '.png'
        dic["images"].append({
            "license": 1,
            "file_name": name,
            "coco_url": "",
            "height": 256,
            "width": 256,
            "date_captured": "2020-05-19 23:03:57",
            "flickr_url": "",
            "id": image_id
        })
        
        dic["annotations"].append({
            "segmentation": [],
            "area": 232,
            "iscrowd": 0,
            "image_id": image_id,
            "bbox": [1, 5, 4, 9],
            "category_id": randint(1, 26),
            "id": anno_id
        })
        anno_id += 1
    with open(json_name, "w") as json_file:
        json.dump(dic, json_file)

In [ ]:
csv_to_coco(pd.read_csv('sample_submission.csv'), 'test.json')

In [ ]:
%cd ASL

/content/drive/My Drive/dacon_cv/ASL
